<a href="https://colab.research.google.com/github/Jo-chana/CoLab_ML_codes/blob/master/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [0]:
%tensorflow_version 1.x 

In [0]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [12]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [14]:
# another way/ session automatically closed when 'with' block finish

with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
print(result)

42


In [15]:
# another way 2/ function creates nodes to initialize all variables when calculation graph is executed

init = tf.global_variables_initializer() # prepare init nodes

with tf.Session() as sess:
  init.run()
  result = f.eval()
print(result)

42


In [16]:
# jupyter or python shell 

sess = tf.InteractiveSession()
init.run()  # automatically designate itself as basic session
result = f.eval()
print(result)
sess.close

42


<bound method InteractiveSession.close of <tensorflow.python.client.session.InteractiveSession object at 0x7f395b64fcc0>>

In [17]:
# making independent calculation graph

graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph)

#initailize: tf.reset_default_graph

True
False


In [18]:
# node lifecycle

w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# In this code w and x are evaluated twice to evaluate y and z
with tf.Session() as sess:
  print(y.eval())
  print(z.eval())

10
15


In [20]:
# w and x are evaluated once in this code

with tf.Session() as sess:
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)

10
15


In [23]:
# tensor calculation

import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
  theta_value = theta.eval()

theta_value

array([[-3.67372932e+01],
       [ 4.37366009e-01],
       [ 9.47520509e-03],
       [-1.08159676e-01],
       [ 6.48537397e-01],
       [-3.84734449e-06],
       [-3.79239232e-03],
       [-4.19136107e-01],
       [-4.32144403e-01]], dtype=float32)

In [28]:
# direct gradient calculation

n_epochs = 1000
learning_rate = 0.1

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')


gradient_mse = 2/m * tf.matmul(tf.transpose(X),error)
# gradient_mse_auto = tf.gradients(mse,[theta])[0]
training = tf.assign(theta, theta-learning_rate*gradient_mse)

# other method using optimizer
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# training = optimizer.minimize(mse)



init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    if epoch%100 == 0:
      print('Epoch',epoch,'MSE:',mse.eval())
    sess.run(training)

  best_theta = theta.eval()

Epoch 0 MSE: 6.227399
Epoch 100 MSE: 4.81175
Epoch 200 MSE: 4.8046083
Epoch 300 MSE: 4.803471
Epoch 400 MSE: 4.803289
Epoch 500 MSE: 4.80326
Epoch 600 MSE: 4.8032546
Epoch 700 MSE: 4.8032537
Epoch 800 MSE: 4.8032537
Epoch 900 MSE: 4.8032537
